# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | arts.ipynb
 |-data\Orcus - Arts.xlsx
 |-output
```

and will take data\Orcus - Arts.xlsx and create a markdown file output\arts.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Arts.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Name,Level,Type,Category,Skill,Completion Time,Components,Cost to Learn,Duration,Description
0,Chapter,0,,,Arcana,,,,,
1,Alarm,1,Abjuration,Incantation,Arcana,1 hour,A tiny bell and a piece of fine silver wire (1...,14 gp,8 hours,You set an alarm against unwanted intrusion. C...
2,Detect Magic,1,Divination,Incantation,Arcana,10 minutes,14 gp,14 gp,10 minutes,"For the duration, you sense the presence of ma..."
3,Floating Disk,1,Crafting,Incantation,Arcana,1 hour,A drop of mercury (14 gp),14 gp,1 hour,"This incantation creates a circular, horizonta..."
4,Identify,1,Divination,Incantation,Arcana,1 hour,A pearl (28 gp),14 gp,Instantaneous,You choose one object that you must touch thro...


In [3]:
#cell 2, examine 5 random arts

data.sample(5)

,Name,Level,Type,Category,Skill,Completion Time,Components,Cost to Learn,Duration,Description
6,Mage's Mark,1,Transmutation,Incantation,Arcana,1 minute,Fresh candle and silver stylus (14 gp),14 gp,Permanent,"Every mage has their own mark, usually an elab..."
62,Bonded Companion,5,Practical,Incantation,Nature,8 hours,An item with special significance to both part...,40 gp,Instantaneous,You perform this incantation over two particip...
86,Craft Trap,1,Schema,Practice,Sleight of Hand,1 hour,*,*,Instantaneous,"Each trap has its own schema, which a characte..."
50,Delve into Collective Memory,3,Divination,Incantation,History,10 minutes,27 gp,27 gp,Instantaneous,You can learn information from the memories of...
93,Party Crasher,7,Interaction,Practice,Streetsmarts,2 hours,104 gp,104 gp,Instantaneous,Any time you encounter a social event you woul...


In [4]:
#cell 3
#we define a function which takes in a row of this art dataframe and returns an md string

def art_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    if row_dict.get('Name') == 'Chapter':
        md = '## {Skill}  '.format(**row_dict)
        return md
    
   
    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
    
    #start with name. md is the string we'll be outputing
    md = "### {Name}  ".format(**row_dict)
    
    #add flavor if it exists
    #'\n' is Python for new line
    md += "\nLevel {Level} {Type} {Category} ({Skill})  ".format(**row_dict)
          
    #this is a complex chunk
    md += """\n\n**Cost to Learn:** {Cost to Learn}  """.format(**row_dict)
    md += """\n**Completion Time:** {Completion Time}  """.format(**row_dict)
    md += """\n**Components:** {Components}  """.format(**row_dict)
    md += """\n**Duration:** {Duration}  """.format(**row_dict)
    
    md += '\n\n{Description}  '.format(**row_dict)
        
    
    #finally we remove the empty tags
    return md.replace('****','')


In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = art_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Arts.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))